In [1]:
adjacency_list = [
    [1,2,3],
    [0,2,3],
    [0,1,3],
    [0,1,2],
    [0,3],
]

edge_list = [
    (0, 1),
    (0, 2),
    (0, 3),
    (1, 0),
    (1, 2),
    (1, 3),
    (2, 0),
    (2, 1),
    (2, 3),
    (3, 0),
    (3, 1),
    (3, 2),
    (4, 0),
    (4, 3),
]

adjacency_matrix = [
    [0,1,1,1,0],
    [1,0,1,1,0],
    [1,1,0,1,0],
    [1,1,1,0,0],
    [1,0,0,1,0],
]

def adj_list_to_mat( adj_list ):
    
    return [ [1 if i in row else 0 for i, _ in enumerate(adj_list) ] 
            for row in adj_list ]

def edgl_to_adjm( edge_list ):
    
    N = max([ n for xy in edge_list for n in xy ]) + 1
    return [ [ 1 if (y,x) in edge_list else 0 for x in range(N) ] for y in range(N) ]
    

def adjl_to_edgl( adj_list ):
    
    return [ (i,j) for i, E in enumerate(adj_list) for j in E ]


def adjm_to_adjl( adj_mat ):
    
    return [ [i for i, v in enumerate(row) if v] 
            for row in adj_mat ]

def symmetricise_mat( adj_mat ):
    
    return [ [ 1 if v or adj_mat[x][y] else 0 
               for x, v in enumerate(row) ] 
             for y, row in enumerate(adj_mat) ]


In [20]:
G1_adjl = [
    [1, 2], # 0
    [2, 3], # 1
    [3, 4], # 2
    [5], # 3
    [5], # 4
    [], # 5
]

G2_edgl = [
    (0,1),
    (0,2),
    (1,4),
    (2,5),
    (5,3),
    (3,6),
    (4,7),
]
G2_adjm = symmetricise_mat( edgl_to_adjm( G2_edgl ) )

G3_edgl = [
    (0,2),
    (0,3),
    (2,4),
    (4,1),
    (1,2),
    (1,0),
    (2,0),
    (5,2),
    (6,5),
    (6,3),
    (6,7),
    (7,4),
    (7,5),
    (7,6),
]
G3_adjm = edgl_to_adjm( G3_edgl )

G4_edgl = [
    (0,2),
    (0,3),
    (2,4),
    (2,3),
    (4,1),
    (5,2),
    (6,5),
    (6,3),
    (6,7),
    (7,4),
    (7,5),
]
G4_adjm = edgl_to_adjm( G4_edgl )


G5_edgl = [
    (0,2),
    (0,3),
    (2,4),
    (4,1),
    (1,2),
    (2,3),
    (5,2),
    (6,3),
    (6,5),
    (6,7),
    (7,4),
    (7,5),
    (7,6),
]
G5_adjm = edgl_to_adjm( G5_edgl )

In [3]:
class Queue:
    def __init__(self):
        self.list = []
        self.front = 0
        
    def push( self, x ):
        self.list.append(x)
        self.set.add(x)
    
    def pop( self ):
        if self.empty():
            return None
        self.front += 1
        return self.list[self.front - 1]
    
    def empty( self ):
        return self.front == len(self.list)

In [13]:
def bfs( G_adjl, s ): # O(V + E)
    
    queue = Queue()
    queue.push( (s,0) )
    
    visited = set([s])
    discover_finishes = { s: [0, None] }
    
    
    while not queue.empty():
        
        current, d = queue.pop()
        print(current, d)
        
        for connection in G_adjl[current]:
            if connection not in visited:
                queue.push( (connection, d + 1) )
                visited.add( connection )

                
class Timer:
    def __init__(self):
        self.time = 0
        
    def inc(self):
        self.time += 1
        
        
def dfs_inner( G_adjm, source_index, visited, timer, group ):
    
    if source_index in visited:
        return
    
    timer.inc()
    start = timer.time

    visited.add(source_index)
    [ dfs_inner( G_adjm, i, visited, timer, group) for i, x in enumerate(G_adjm[source_index]) if x ]
    
    timer.inc()
    finish = timer.time
    group.append( ( source_index, start, finish ) )
    

def dfs( G_adjm, source_order=None ):
    
    visited = set()
    timer = Timer()
    groups = []
    
    for i in (source_order or range(len(G_adjm))):
        groups.append([])
        dfs_inner( G_adjm, i, visited, timer, groups[-1] )
        
    return groups

In [27]:
dfs( G4_adjm )

[[(3, 3, 4), (1, 6, 7), (4, 5, 8), (2, 2, 9), (0, 1, 10)],
 [],
 [],
 [],
 [],
 [(5, 11, 12)],
 [(7, 14, 15), (6, 13, 16)],
 []]

In [25]:
def strongly_connected( G_adjm ):
    
    nodes_forward = [ i for group in dfs( G_adjm ) for i, start, finish in group ]
    groups_backward = [ node for node in dfs( G_adjm, source_order=nodes_forward ) ]
    
    return [ [ i for i, s, f in group ] for group in groups_backward if group  ]

In [26]:
strongly_connected( G5_adjm )

[[3], [4, 2, 1], [0], [5], [6, 7]]

In [24]:
G_adjl = [
    [(1,6),(2,1),(3,1)],
    [(0,1),(2,1),(3,1)],
    [(0,1),(1,1),(3,1)],
    [(0,1),(1,1),(2,1)]
]


def mst( G_adjl, source_index=0 ):
    
    N = len( G_adjl )
    frontier = [ source_index ]
    
    spanning_tree = []
    
    
    while True:
        
        candidates = [
            (i,j,w) for i, connections in enumerate([G_adjl[i] for i in frontier])
            for (j,w) in connections
            if j not in frontier and i != j
        ]
        
        if not candidates:
            return spanning_tree
        
        print(candidates)
        
        min_edge = min( candidates, key=lambda e: e[2] )
        
        spanning_tree.append( min_edge )
        frontier.append( min_edge[1] )

In [25]:
mst( G_adjl )

[(0, 1, 6), (0, 2, 1), (0, 3, 1)]
[(0, 1, 6), (0, 3, 1), (1, 3, 1)]
[(0, 1, 6), (2, 1, 1)]


[(0, 2, 1), (0, 3, 1), (2, 1, 1)]